In [ ]:
import numpy as np
import cv2
from tqdm import tqdm
from PIL import Image
import os
import gc
import skimage.io

In [ ]:
MAXIMUM_SIZE = 1024

def crop_white(image):
    if np.unique(image).shape[0] != 1:
        #CROPPING WHITE SPACE
        white = np.array([255, 255, 255])
        mask = np.abs(image - white).sum(axis=2) < 5

        # Find the bounding box of those pixels
        coords = np.array(np.nonzero(~mask))
        top_left = np.min(coords, axis=1)
        bottom_right = np.max(coords, axis=1)

        image = image[top_left[0]:bottom_right[0],
                    top_left[1]:bottom_right[1]]

        gc.collect()
    return image

def adjust_size(vv):

    h, w, c = vv.shape

    if h > MAXIMUM_SIZE and w <= MAXIMUM_SIZE:
        vv = cv2.resize(vv, (w, MAXIMUM_SIZE), interpolation=cv2.INTER_AREA)
        h, w, c = vv.shape

    elif h <= MAXIMUM_SIZE and w > MAXIMUM_SIZE:
        vv = cv2.resize(vv, (MAXIMUM_SIZE, h), interpolation=cv2.INTER_AREA)
        h, w, c = vv.shape


    elif h > MAXIMUM_SIZE and w > MAXIMUM_SIZE:
        vv = cv2.resize(vv, (MAXIMUM_SIZE, MAXIMUM_SIZE), interpolation=cv2.INTER_AREA)
        h, w, c = vv.shape
        
    tb = MAXIMUM_SIZE - h
    t = tb // 2
    b = tb - t

    lr = MAXIMUM_SIZE - w
    l = lr // 2
    r = lr - l
    
    vv = cv2.copyMakeBorder(vv.copy(),t,b,l,r,cv2.BORDER_CONSTANT,value=[255, 255, 255])
    gc.collect()
    return vv

def make_tiles(vv):
    tile_size = 1024
    h, w, c = vv.shape
    img = vv.reshape(h // tile_size, tile_size, w // tile_size, tile_size,3)
    img = img.transpose(0,2,1,3,4).reshape(-1, tile_size, tile_size,3)
    return img

def encode_image(vv, MAX_PER_CHANNEL = 10):
    DIV_FACTOR = 255 / (MAX_PER_CHANNEL - 1)
    vv = vv / DIV_FACTOR
    vv = vv.astype('ulonglong')
    pows = []
    k = 0
    while k < vv.shape[0]:
        power = MAX_PER_CHANNEL ** k
        vv[k, ] = vv[k, ] * power
        k += 1
    vv = np.sum(vv, axis=0)
    return vv

def decode_image(x, MAX_PER_CHANNEL = 10, SMOOTH=True):
    x = x.astype('ulonglong')
    DIV_FACTOR = 255 / (MAX_PER_CHANNEL - 1)
    vv = []
    k = 0
    while k < 16:
        v = x % MAX_PER_CHANNEL
        vv.append(v)
        x = x // MAX_PER_CHANNEL
        
        k += 1
    vv = np.stack(vv)
    vv = vv * DIV_FACTOR
    if SMOOTH:
        k = 0
        while k < vv.shape[0]:
            vv[k, ] = smooth_decoded(vv[k, ])
            k += 1
    return vv

def smooth_decoded(x):
    white = np.array([255, 255, 255])
    mask = np.abs(x - white).sum(axis=2) < 120
    x[mask] = 255
    return x

In [ ]:
import os

In [ ]:
TRAIN = '../input/prostate-cancer-grade-assessment/train_images/'
imgs = os.listdir(TRAIN)

In [ ]:
os.mkdir('train')

In [ ]:
x = skimage.io.MultiImage(os.path.join(TRAIN,imgs[6]))[0]

In [ ]:
cropped = crop_white(x)
x.shape, cropped.shape

# Original Image

In [ ]:
from PIL import Image
cropped = Image.fromarray(cropped)
crop_size = 4096
width, height = cropped.size   # Get dimensions
left = (width - crop_size)/2
top = (height - crop_size)/2
right = (width + crop_size)/2
bottom = (height + crop_size)/2
# Crop the center of the image
cropped = cropped.crop((left, top, right, bottom))

In [ ]:
cropped

# Make Tiles

In [ ]:
tiles = make_tiles(np.array(cropped))

In [ ]:
tiles.shape

# Some Tiles

In [ ]:
Image.fromarray(tiles[0].astype('uint8'))

In [ ]:
Image.fromarray(tiles[8].astype('uint8'))

# Fold Tiles, Encode and Compress

In [ ]:
encoded = encode_image(tiles)

In [ ]:
np.savez_compressed('train/encoded.npz', encoded)

# Decompress, Decode and UnFold Tiles

In [ ]:
decoded = decode_image(encoded)

In [ ]:
decoded.shape

In [ ]:
Image.fromarray(decoded[0].astype('uint8'))

In [ ]:
Image.fromarray(decoded[8].astype('uint8'))